In [ ]:
# # Install if not already installed
# !pip3 install datasets --quiet
# !pip3 install nltk --quiet

# Imports
from datasets import load_dataset
from nltk.tokenize import sent_tokenize
import nltk
nltk.download("punkt")

# --- 1. Load a small sample from the Wikipedia dataset ---
wiki = load_dataset("wikipedia", "20220301.en", split='train[:1%]')
print("\n📘 Sample Wikipedia article (raw text):\n")
print(wiki[0]['text'][:1000])  # Show the first 1000 characters

# --- 2. Sentence tokenize the first article ---
sentences = sent_tokenize(wiki[0]['text'])
print("\n📌 Tokenized sentences from the first article:\n")
for i, sent in enumerate(sentences[:5]):
    print(f"{i+1}. {sent}")

# --- 3. Load WikiText dataset (cleaner format) ---
wikitext = load_dataset("wikitext", "wikitext-103-raw-v1", split="train[:1%]")
print("\n📙 Sample WikiText lines:\n")
for i, line in enumerate(wikitext[:5]['text']):
    print(f"{i+1}. {line}")
